## 1. Two Sum

In [ ]:
;;; 使用散列表
(defun two-sum (nums target)
  (let ((hash-table (make-hash-table)))
    (loop for i below (length nums) do
         (let* ((key (- target (nth i nums)))
                (value (gethash key hash-table)))
           (if value
               (return (list value i))
               (setf (gethash (nth i nums) hash-table) i))))))

## 2. Add Two Numbers

In [ ]:
;;; 对两个链表进行遍历，重复的代码有点多
(defun add-two-numbers (l1 l2)
  (let ((res nil)
        (quotient)
        (remainder 0))
    (loop while (and l1 l2) do
         (setf (values remainder quotient)
               (floor (+ (car l1) (car l2) remainder) 10))
         (push quotient res)
         (setf l1 (cdr l1))
         (setf l2 (cdr l2)))
    (loop while l1 do
         (setf (values remainder quotient)
               (floor (+ (car l1) remainder) 10))
         (push quotient res)
         (setf l1 (cdr l1)))
    (loop while l2 do
         (setf (values remainder quotient)
               (floor (+ (car l2) remainder) 10))
         (push quotient res)
         (setf l2 (cdr l2)))
    (if (> remainder 0)
        (reverse (push remainder res))
        (reverse res))))

In [ ]:
;;; 将重复的 loop while 部分进行抽取，勉强弄成这样
(defmacro lw ((lst &optional (another '(0))) &body body)
  `(loop while (and ,lst ,another) do
        (setf (values remainder quotient)
              (floor (+ (car ,lst)
                        (car ,another)
                        remainder) 10))
        (push quotient res)
        ,@body))

(defun add-two-numbers (l1 l2)
  (let ((res nil)
        (quotient)
        (remainder 0))
    (lw (l1 l2)
        (setf l1 (cdr l1))
        (setf l2 (cdr l2)))
    (lw (l1 '(0))
        (setf l1 (cdr l1)))
    (lw (l2 '(0))
        (setf l2 (cdr l2)))
    (if (> remainder 0)
        (reverse (push remainder res))
        (reverse res))))

## 3. Longest Substring Without Repeating Characters

In [ ]:
;;; 使用关联表 alist 存储出现字符与对应的索引
;;; 函数 ASSOC 接受一个键和一个 alist 并返回第一个 CAR 匹配该键的点对单元
;;; 双指针进行查找
(defun length-of-longest-substring (s)
  (let ((alist nil)
        (longest-length 0)
        (p -1))
    (loop
       for c across s
       for index from 0 to (length s) do
         (let ((cell (assoc c alist))
               (size (- index p)))
           (cond
             (cell (setf p (cdr cell)))
             ((> size longest-length) (setf longest-length size))))
         (push (cons c index) alist))
    longest-length))

## 4. Median of Two Sorted Arrays

In [ ]:
;;; 遍历两个数组并将数据放入新的排序好的数组中，再计算
;;; multiple-value-bind 宏接受多重返回值
(defun find-median-sorted-arrays (nums1 nums2)
  (let ((lst nil)
        (len (+ (length nums1) (length nums2))))
    (loop while (and nums1 nums2) do
         (if (< (car nums1) (car nums2))
             (push (pop nums1) lst)
             (push (pop nums2) lst)))
    (setf lst (reverse lst))
    (nconc lst nums1 nums2)
    (multiple-value-bind (q r) (floor len 2)
      (if (zerop r)
          (/ (+ (nth q lst) (nth (1- q) lst)) 2)
          (nth q lst)))))

In [ ]:
;;; 将两个数组合并后排序，再计算
(defun find-median-sorted-arrays-1 (nums1 nums2)
  (let* ((lst (sort (append nil nums1 nums2) #'<))
         (len (length lst)))
    (multiple-value-bind (q r) (floor len 2)
      (if (zerop r)
          (/ (+ (nth q lst) (nth (1- q) lst)) 2)
          (nth q lst)))))


## 5. Longest Palindromic Substring

In [ ]:
;;; 双指针进行查找，max-length 存储最大长度，start 存在最大长度开始位置。
;;; 遍历过程中，判断当前指针与前 max-length + 1 或 max-length + 2 中
;;; 的字符串是否为回文序列。
;;; 使用宏抽取重复，感觉宏被用歪了 :)
(defmacro increase-length (step)
  `(let* ((in-start (- i max-length ,step -1))
          (sub (subseq s in-start (+ i 1))))
     (when (string= sub (reverse sub))
       (incf max-length ,step)
       (setf start in-start))))

(defun longest-palindrome (s)
  (let ((size (length s))
        (start 0)
        (max-length 1))
    (if (string= s (reverse s))
        (return-from longest-palindrome s))
    (loop for i from 1 below size do
         (if (null (if (>= (- i max-length 1) 0)
                       (increase-length 2)))
             (increase-length 1)))
    (subseq s start (+ start max-length))))


## 6. ZigZag Conversion

In [ ]:
;;; 找到规律后按规律写出代码
(defun convert (s num-rows)
  (if (or (= 1 num-rows)
          (> num-rows (length s)))
      s
      (let* ((length-s (length s))
             (res (make-array length-s
                              :fill-pointer 0
                              :element-type 'character))
             (step (* 2 (- num-rows 1))))
        (loop for n below num-rows do
             (do* ((idx n (+ idx step))
                   (ex-idx (+ idx (- step (* 2 n))) (+ idx (- step (* 2 n)))))
                  ((>= idx length-s))
               (vector-push (elt s idx) res)
               (if (and (< ex-idx length-s)
                        (not (zerop n))
                        (not (= idx ex-idx)))
                   (vector-push (elt s ex-idx) res))))
        res)))

## 7. Reverse Integer

In [ ]:
;;; 转换成字符串 :)
(defun reverse-int (x)
  (if (>= x 0)
      (+ (parse-integer (reverse (write-to-string x))))
      (- (parse-integer (reverse (write-to-string (abs x)))))))

## 8. String to Integer (atoi)

In [ ]:
;;; 使用数组而不是列表是因为 vector-push 能把元素添加到尾部
(defun my-atoi (str)
  (let ((str (string-trim " " str))
        (digit (make-array 12
                           :fill-pointer 0
                           :element-type 'character)))
    (if (zerop (length str))
        (return-from my-atoi 0)
        (vector-push (elt str 0) digit))
    (loop for s across (subseq str 1) do
         (if (digit-char-p s)
             (vector-push s digit)
             (return)))
    (handler-case (let ((res (parse-integer digit))
                        (limit (expt 2 31)))
                    (cond ((> res (1- limit))
                           (1- limit))
                          ((< res (- limit))
                           (- limit))
                          (t res)))
      (error () 0))))

## 9. Palindrome Number

In [ ]:
(defun is-palindrome (x)
  (string= (write-to-string x)
           (reverse (write-to-string x))))

## 10. Regular Expression Matching

In [ ]:
;;; 动态规划-备忘录进行匹配
(defun is-match (s p)
  (let ((memo (make-hash-table :test 'equal)))
    (defun dp (str pattern)
      (let ((key (list str pattern)))
        (multiple-value-bind (value present) (gethash key memo)
          (if present
              value
              (progn
                (cond ((string= pattern "")
                       (setf (gethash key memo) (string= str "")))
                      ((string= str "")
                       (setf (gethash key memo)
                             (and (eql (position #\* pattern) 1)
                                  (dp str (subseq pattern 2)))))
                      ((eql (position #\* pattern) 1)
                       (setf (gethash key memo)
                             (or (dp str (subseq pattern 2))
                                 (dp (subseq str 1) pattern))))
                      ((or (string= (elt pattern 0) #\.)
                           (string= (elt pattern 0) (elt str 0)))
                       (setf (gethash key memo)
                             (dp (subseq str 1) (subseq pattern 1))))
                      (t
                       (setf (gethash key memo) nil)))
                (gethash key memo))))))
    (dp s p)))

## 11. Container With Most Water

In [ ]:
;;; 双指针
(defmacro shorten (short i step)
  `(progn
     (incf ,i ,step)
     (setf area (* ,short (- l r)))
     (loop while (and (< r l) (< (nth ,i height) ,short)) do
          (incf ,i ,step))))

(defun max-area (height)
  (let ((r 0)
        (l (1- (length height)))
        (area 0)
        (max-area 0))
    (loop while (< r l) do
         (if (< (nth r height) (nth l height))
             (shorten (nth r height) r 1)
             (shorten (nth l height) l -1))
         (if (> area max-area)
             (setf max-area area)))
    max-area))

## 12. Integer to Roman

In [ ]:
;;; :)
;;; 自己实现的话，将个十百千位拆出来再转换
(defun int-to-roman (num)
  (format nil "~@R" num))

## 13. Roman to Integer

In [ ]:
;;; 每位字符转换为对应的数值，再计算
(defun roman-to-integer (s)
  (let* ((roman "IVXLCDM")
         (nums '(1 5 10 50 100 500 1000))
         (roman-nums (loop for c across s
                        collect (nth (position c roman) nums))))
    (loop for (a b) on roman-nums
       sum (if (and b (< a b)) (- a) a))))

## 14. Longest Common Prefix

In [ ]:
;;; 按顺序取出所有对应字符，再判断是否全部相同
(defun longest-common-prefix (strs)
  (if (not strs)
      (return-from longest-common-prefix ""))
  (let* ((chars (apply #'map 'list #'list strs))
         (size (length chars)))
    (print chars)
    (loop
       for ch in chars
       for i to size do
         (if (not (= (length (remove-duplicates ch)) 1))
             (return-from longest-common-prefix (subseq (first strs) 0 i))))
    (subseq (first strs) 0 size)))

## 15. 3Sum

In [ ]:
(defun three-sum (nums)
  (let ((table (make-hash-table))
        (neg-nums nil)
        (pos-nums nil)
        (res nil))
    (loop for n in nums do
         (let ((value (gethash n table)))
           (if value
               (setf (gethash n table) (1+ value))
               (setf (gethash n table) 1))))
    (loop for k being the hash-keys in table do
         (if (< k 0)
             (push k neg-nums)
             (push k pos-nums)))
    (setf neg-nums (sort neg-nums '<))
    (setf pos-nums (sort pos-nums '<))
    (loop for n in neg-nums do
         (loop for p in pos-nums do
              (let* ((balance (- 0 n p))
                     (value (gethash balance table)))
                (if (and value
                         (or (and (or (= balance n) (= balance p))
                                  (> value 1))
                             (and (< n balance)
                                  (< balance p))))
                    (push (list n balance p) res)))))
    res))

## 16. 3Sum Closest

In [ ]:
(defun three-sum-closest (nums target)
  (let ((size (length nums))
        (nums (sort nums #'<))
        (closest nil))
    (loop for i to (- size 3) do
         (let* ((a (1+ i))
                (b (1- size))
                (min (+ (nth i nums)
                        (nth a nums)
                        (nth (1+ a) nums)))
                (max (+ (nth i nums)
                        (nth (1- b) nums)
                        (nth b nums))))
           (cond
             ((< max target) (push max closest))
             ((> min target) (push min closest))
             (t (loop while (< a b) do
                     (let ((sum (+ (nth i nums)
                                   (nth a nums)
                                   (nth b nums))))
                       (push sum closest)
                       (cond
                         ((< sum target) (incf a))
                         ((> sum target) (decf b))
                         (t (return-from three-sum-closest target)))))))))
    (first (sort closest #'(lambda (x y)
                             (< (abs (- x target))
                                (abs (- y target))))))))

## 17. Letter Combinations of a Phone Number

In [ ]:
(defun letter-combinations (digits)
  (let* ((letters (pairlis
                   '(#\2 #\3 #\4 #\5 #\6 #\7 #\8 #\9)
                   '("abc" "def" "ghi" "jkl" "mno" "pqrs" "tuv" "wxyz")))
         (res (loop for c across
                   (cdr (assoc (elt digits 0) letters)) collect (string c))))
    (loop for d across (subseq digits 1) do
         (let ((strs nil))
           (loop for a across (cdr (assoc d letters)) do
                (loop for r in res do
                     (push (concatenate 'string r `(,a)) strs)))
           (setf res strs)))
    res))

## 18. 4Sum

In [ ]:
(defun n-sum (nums target n &optional (result nil) (results nil))
  (if (= n 2)
      (let ((start 0)
            (end (1- (length nums))))
        (loop while (< start end) do
             (let* ((a (nth start nums))
                    (b (nth end nums))
                    (s (+ a b)))
               (cond
                 ((= s target)
                  (push (append result (list a b)) results)
                  (incf start)
                  (decf end)
                  (loop while (and (< start end)
                                   (= (nth start nums)
                                      (nth (1- start) nums)))
                     do (incf start))
                  (loop while (and (< start end)
                                   (= (nth end nums)
                                      (nth (1+ end) nums)))
                     do (decf end)))
                 ((< s target)
                  (incf start))
                 (t
                  (decf end))))))
      ;; n > 2
      (loop for i to (- (length nums) n) do
           (if (and (not (zerop i))
                    (= (nth i nums) (nth (1- i) nums)))
               nil
               (setf results (n-sum (subseq nums (1+ i))
                                      (- target (nth i nums))
                                      (1- n)
                                      (append result (list (nth i nums)))
                                      results)))))
  results)

(defun four-sum (nums target)
  (n-sum (sort nums #'<) target 4))

## 19. Remove Nth Node From End of List

In [ ]:
(defun remove-nth-from-end (head n)
  (let ((i (- (length head) n)))
    (append (subseq head 0 i)
            (subseq head (1+ i)))))

## 20. Valid Parentheses

In [ ]:
(defun is-valid (s)
  (let ((res nil)
        (pair (pairlis '(#\) #\] #\}) '(#\( #\[ #\{))))
    (loop for c across s do
         (if (and res
                  (eql (first res) (cdr (assoc c pair))))
             (pop res)
             (push c res)))
    (not res)))

## 21. Merge Two Sorted Lists

In [ ]:
;;; :)
;;; 若是用 car 和 cdr 呢
(defun merge-two-lists (l1 l2)
  (sort (append l1 l2) #'<))

## 22. Generate Parentheses

In [ ]:
;;; 动态规划
(let ((alist (pairlis '(0 1) (list nil '("()")))))
  (defun generate-parenthesis (n)
    (cond
      ((assoc n alist)
       (cdr (assoc n alist)))
      (t
       (let ((res nil))
         (loop for i from 1 below (1- n) do
              (loop for a in (generate-parenthesis i) do
                   (loop for b in (generate-parenthesis (- n 1 i)) do
                        (push (concatenate 'string "(" a ")" b)
                              res))))
         (loop for x in (generate-parenthesis (1- n)) do
              (push (concatenate 'string "(" x ")") res)
              (push (concatenate 'string "()" x) res))
         (push (cons n (remove-duplicates res :test #'string=))
               alist)
         (cdr (assoc n alist)))))))

## 23. Merge k Sorted Lists

In [ ]:
;;; 常规思路应该是先从合并两个有序列表开始
(defun merge-k-lists (lists)
  (sort (apply #'append lists) #'<))

## 24. Swap Nodes in Pairs

In [ ]:
(defun swap-pairs (head)
  (loop for i below (floor (length head) 2) do
       (rotatef (nth (* i 2) head) (nth (1+ (* i 2)) head)))
  head)

## 25. Reverse Nodes in k-Group

In [ ]:
(defun reverse-k-group (lst k)
  (let ((res (copy-list lst)))
    (if (= k 1)
        (return-from reverse-k-group res))
    (loop for end from k to (length lst) by k do
         (setf (subseq res (- end k) end)
               (reverse (subseq res (- end k) end))))
    res))